In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\Vrushali Shah\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv("../Data/training.csv")

In [3]:
df.didPurchase = (df.didPurchase)*1
df.doRecommend = (df.doRecommend)*1

In [4]:
df['didPurchase'] = df['didPurchase'].fillna(1)
df['doRecommend'] = df['doRecommend'].fillna(1)

In [5]:
training_set = df[['username','rating','doRecommend','didPurchase']]
#training_set=training_set.iloc[0:10000,:]
training_set.shape

(56802, 4)

In [6]:
df = pd.read_csv("../Data/test.csv")

In [7]:
df.didPurchase = (df.didPurchase)*1
df.doRecommend = (df.doRecommend)*1

In [8]:
df['didPurchase'] = df['didPurchase'].fillna(1)
df['doRecommend'] = df['doRecommend'].fillna(1)

In [9]:
test_set = df[['username','rating','doRecommend','didPurchase']]
#test_set=test_set.iloc[0:10000,:]
test_set.shape

(14201, 4)

In [10]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
#training_set['rating'] = sc.fit_transform(training_set[['rating']])
#print(training_set['didPurchase'])
#training_set['didPurchase'] = sc.fit_transform(training_set[['didPurchase']])
#print(training_set['didPurchase'])

## Part A

In [11]:
# Training Parameters
learning_rate = 0.0001
training_steps = 200
batch_size = 300

# Network Parameters
num_input = 3 # MNIST data input (img shape: 28*28)
timesteps = 1 # timesteps
num_hidden = 100 # hidden layer num of features
num_classes = 2 # MNIST total classes (0-9 digits)



In [12]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input], name="x")
Y = tf.placeholder("float", [None, num_classes], name="label")

In [13]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]),name="W")
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]),name="B")
}

In [14]:
#tf.summary.histogram("weights", weights)
#tf.summary.histogram("biases", biases)

In [15]:
def RNN(x, weights, biases):
   
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    #lstm_cell=rnn.MultiRNNCell([rnn.BasicLSTMCell(num_hidden),rnn.BasicLSTMCell(num_hidden)])
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [16]:
logits = RNN(X, weights, biases)

In [17]:
# Start training
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()
def start_training(flag=True):
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./log_summaries")
    with tf.Session() as sess:
        # Initialize the variables (i.e. assign their default value)
        init = tf.global_variables_initializer()
        # Run the initializer
        sess.run(init)
        display_step = training_steps/10
        
        for step in range(1, training_steps+1):
            #X_train, X_test, y_train, y_test = train_test_split(training_set[['rating','didPurchase']], training_set[['doRecommend']], test_size=0.2)
            #batch_x, batch_y = X_train, enc.fit_transform(y_train).toarray()[0:batch_size,:]
            batch=training_set.sample(batch_size)
            batch_x, batch_y=batch[['username','rating','didPurchase']], enc.fit_transform(batch[['doRecommend']]).toarray()
            # Reshape data to get 28 seq of 28 elements
            batch_x = batch_x.values.reshape((batch_size, timesteps, num_input))
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            if (step % display_step == 0 or step == 1) and flag:
                # Calculate batch loss and accuracy
                loss, acc, s = sess.run([loss_op, accuracy, summ], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                writer.add_summary(s, step)
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))
            elif training_steps<=20 and flag:
                loss, acc, s = sess.run([loss_op, accuracy, summ], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                writer.add_summary(s, step)
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

        if flag:
            print("Optimization Finished!")

        # Calculate accuracy for 128 mnist test images
        test_len = 128
        test_data =test_set[['username','rating','didPurchase']].values.reshape((-1, timesteps, num_input))
        test_label = enc.fit_transform(test_set[['doRecommend']]).toarray()#[:test_data.shape[0],:]
        print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))
        writer.add_graph(sess.graph)
        sess.close()

In [18]:
# epochs = 200
# batch size = 300
# num of hidden features = 100
# activation function = Softmax
# cost function = cross entropy
# gradient estimator = RMS

In [19]:
prediction = tf.nn.tanh(logits)
tf.summary.histogram("tanh", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_tanh_RMS", accuracy)

<tf.Tensor 'accuracy_tanh_RMS:0' shape=() dtype=string>

In [20]:

start_training()

Step 1, Minibatch Loss= 3.3334, Training Accuracy= 0.053
Step 20, Minibatch Loss= 3.3035, Training Accuracy= 0.077
Step 40, Minibatch Loss= 3.3000, Training Accuracy= 0.070
Step 60, Minibatch Loss= 3.2752, Training Accuracy= 0.083
Step 80, Minibatch Loss= 3.2869, Training Accuracy= 0.053
Step 100, Minibatch Loss= 3.2872, Training Accuracy= 0.047
Step 120, Minibatch Loss= 3.2421, Training Accuracy= 0.080
Step 140, Minibatch Loss= 3.2371, Training Accuracy= 0.067
Step 160, Minibatch Loss= 3.2340, Training Accuracy= 0.063
Step 180, Minibatch Loss= 3.2086, Training Accuracy= 0.077
Step 200, Minibatch Loss= 3.2127, Training Accuracy= 0.060
Optimization Finished!
Testing Accuracy: 0.068023376


## Part B

In [21]:
# epochs = 200
# batch size = 300
# num of hidden features = 100
# activation function = sigmoid
# cost function = sigmoid cross entropy
# gradient estimator = RMS

In [22]:
prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("sigmoid", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_sigmoid_RMS", accuracy)

<tf.Tensor 'accuracy_sigmoid_RMS:0' shape=() dtype=string>

In [23]:
start_training()

Step 1, Minibatch Loss= 1.1779, Training Accuracy= 0.097
Step 20, Minibatch Loss= 1.1952, Training Accuracy= 0.063
Step 40, Minibatch Loss= 1.1532, Training Accuracy= 0.093
Step 60, Minibatch Loss= 1.1475, Training Accuracy= 0.067
Step 80, Minibatch Loss= 1.1106, Training Accuracy= 0.090
Step 100, Minibatch Loss= 1.0989, Training Accuracy= 0.073
Step 120, Minibatch Loss= 1.0652, Training Accuracy= 0.093
Step 140, Minibatch Loss= 1.0511, Training Accuracy= 0.077
Step 160, Minibatch Loss= 1.0270, Training Accuracy= 0.080
Step 180, Minibatch Loss= 1.0101, Training Accuracy= 0.070
Step 200, Minibatch Loss= 0.9808, Training Accuracy= 0.087
Optimization Finished!
Testing Accuracy: 0.068023376


In [24]:
# epochs = 200
# batch size = 300
# num of hidden features = 100
# activation function = relu
# cost function = sigmoid cross entropy
# gradient estimator = RMS

In [25]:
prediction = tf.nn.relu(logits)
tf.summary.histogram("relu", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_relu_RMS", accuracy)

<tf.Tensor 'accuracy_relu_RMS:0' shape=() dtype=string>

In [26]:
start_training()

Step 1, Minibatch Loss= 1.2986, Training Accuracy= 0.057
Step 20, Minibatch Loss= 1.2578, Training Accuracy= 0.057
Step 40, Minibatch Loss= 1.1759, Training Accuracy= 0.093
Step 60, Minibatch Loss= 1.1635, Training Accuracy= 0.047
Step 80, Minibatch Loss= 1.0762, Training Accuracy= 0.090
Step 100, Minibatch Loss= 1.0492, Training Accuracy= 0.063
Step 120, Minibatch Loss= 0.9934, Training Accuracy= 0.077
Step 140, Minibatch Loss= 0.9509, Training Accuracy= 0.070
Step 160, Minibatch Loss= 0.9127, Training Accuracy= 0.053
Step 180, Minibatch Loss= 0.8670, Training Accuracy= 0.057
Step 200, Minibatch Loss= 0.8238, Training Accuracy= 0.060
Optimization Finished!
Testing Accuracy: 0.068023376


Activation functions used relu, tanh, sigmoid, softmax

The accuracy achieved with test data with all four activation function was quite good accuracy of ~ 93%

## Part C

In [27]:
# epochs = 200
# batch size = 300
# num of hidden features = 100
# activation function = sigmoid
# cost function = mean_squared_error
# gradient estimator = RMS

In [28]:
prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("MSE", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.losses.mean_squared_error(logits,prediction)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_MSE_RMS", accuracy)

<tf.Tensor 'accuracy_MSE_RMS:0' shape=() dtype=string>

In [29]:
start_training()

Step 1, Minibatch Loss= 8.7318, Training Accuracy= 0.067
Step 20, Minibatch Loss= 8.4524, Training Accuracy= 0.043
Step 40, Minibatch Loss= 8.2108, Training Accuracy= 0.060
Step 60, Minibatch Loss= 7.9773, Training Accuracy= 0.063
Step 80, Minibatch Loss= 7.7481, Training Accuracy= 0.070
Step 100, Minibatch Loss= 7.5229, Training Accuracy= 0.047
Step 120, Minibatch Loss= 7.3012, Training Accuracy= 0.073
Step 140, Minibatch Loss= 7.0834, Training Accuracy= 0.073
Step 160, Minibatch Loss= 6.8693, Training Accuracy= 0.053
Step 180, Minibatch Loss= 6.6590, Training Accuracy= 0.040
Step 200, Minibatch Loss= 6.4524, Training Accuracy= 0.043
Optimization Finished!
Testing Accuracy: 0.06710795


In [30]:
# epochs = 200
# batch size = 300
# num of hidden features = 100
# activation function = sigmoid
# cost function = hinge loss
# gradient estimator = RMS

In [31]:
prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("sigmoid", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.losses.hinge_loss(logits,prediction)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_sigmoid_RMS", accuracy)

<tf.Tensor 'accuracy_sigmoid_RMS_1:0' shape=() dtype=string>

In [32]:
start_training()

Step 1, Minibatch Loss= 0.8509, Training Accuracy= 0.903
Step 20, Minibatch Loss= 0.8218, Training Accuracy= 0.920
Step 40, Minibatch Loss= 0.7846, Training Accuracy= 0.937
Step 60, Minibatch Loss= 0.7695, Training Accuracy= 0.940
Step 80, Minibatch Loss= 0.7644, Training Accuracy= 0.947
Step 100, Minibatch Loss= 0.7592, Training Accuracy= 0.940
Step 120, Minibatch Loss= 0.7540, Training Accuracy= 0.930
Step 140, Minibatch Loss= 0.7488, Training Accuracy= 0.937
Step 160, Minibatch Loss= 0.7469, Training Accuracy= 0.960
Step 180, Minibatch Loss= 0.7386, Training Accuracy= 0.950
Step 200, Minibatch Loss= 0.7335, Training Accuracy= 0.967
Optimization Finished!
Testing Accuracy: 0.9329625


Tested RNN with MSE and hinge loss both gave ~ 93% accuracy. Thus, there wasn't any effect with change in loss function

## Part D

In [33]:
# epochs = 200
# batch size = 300
# num of hidden features = varying
# activation function = sigmoid
# cost function = hinge loss
# gradient estimator = RMS

In [34]:
prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("sigmoid", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.losses.hinge_loss(logits,prediction)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_MSE_RMS", accuracy)

<tf.Tensor 'accuracy_MSE_RMS_1:0' shape=() dtype=string>

In [35]:
for i in range(1000,5500,1000):
    print("Epochs: %i" % i)
    training_steps=i
    start_training(False)
    print(30 *"-")

Epochs: 1000
Testing Accuracy: 0.9329625
------------------------------
Epochs: 2000
Testing Accuracy: 0.93218786
------------------------------
Epochs: 3000
Testing Accuracy: 0.06781212
------------------------------
Epochs: 4000
Testing Accuracy: 0.93275124
------------------------------
Epochs: 5000
Testing Accuracy: 0.06703753
------------------------------


## Part E

In [36]:
training_steps=1000

In [37]:
# epochs = 100
# batch size = 300
# num of hidden features = 1000
# activation function = sigmoid
# cost function = hinge loss
# gradient estimator = Adagrad

In [38]:
prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("sigmiod", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits, labels=Y))
    optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_sigmoid_Adagrad", accuracy)

<tf.Tensor 'accuracy_sigmoid_Adagrad:0' shape=() dtype=string>

In [39]:
start_training()

Step 1, Minibatch Loss= 0.9417, Training Accuracy= 0.917
Step 100, Minibatch Loss= 0.8710, Training Accuracy= 0.943
Step 200, Minibatch Loss= 0.8722, Training Accuracy= 0.940
Step 300, Minibatch Loss= 0.9103, Training Accuracy= 0.920
Step 400, Minibatch Loss= 0.9000, Training Accuracy= 0.923
Step 500, Minibatch Loss= 0.8442, Training Accuracy= 0.947
Step 600, Minibatch Loss= 0.8493, Training Accuracy= 0.943
Step 700, Minibatch Loss= 0.9124, Training Accuracy= 0.913
Step 800, Minibatch Loss= 0.8363, Training Accuracy= 0.947
Step 900, Minibatch Loss= 0.9071, Training Accuracy= 0.913
Step 1000, Minibatch Loss= 0.8607, Training Accuracy= 0.933
Optimization Finished!
Testing Accuracy: 0.9329625


In [40]:
# epochs = 100
# batch size = 300
# num of hidden features = 1000
# activation function = sigmoid
# cost function = hinge loss
# gradient estimator = AdaDelta

In [41]:
prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("sigmoid", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits, labels=Y))
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_sigmoid_AdaDelta", accuracy)

<tf.Tensor 'accuracy_sigmoid_AdaDelta:0' shape=() dtype=string>

In [42]:
start_training()

Step 1, Minibatch Loss= 1.4375, Training Accuracy= 0.933
Step 100, Minibatch Loss= 1.4864, Training Accuracy= 0.917
Step 200, Minibatch Loss= 1.4570, Training Accuracy= 0.927
Step 300, Minibatch Loss= 1.5547, Training Accuracy= 0.893
Step 400, Minibatch Loss= 1.4471, Training Accuracy= 0.930
Step 500, Minibatch Loss= 1.3982, Training Accuracy= 0.947
Step 600, Minibatch Loss= 1.3786, Training Accuracy= 0.953
Step 700, Minibatch Loss= 1.3785, Training Accuracy= 0.953
Step 800, Minibatch Loss= 1.4077, Training Accuracy= 0.943
Step 900, Minibatch Loss= 1.4468, Training Accuracy= 0.930
Step 1000, Minibatch Loss= 1.4142, Training Accuracy= 0.940
Optimization Finished!
Testing Accuracy: 0.9329625


In [43]:
# epochs = 200
# batch size = 300
# num of hidden features = 1000
# activation function = sigmoid
# cost function = hinge loss
# gradient estimator = GradientDescent

In [44]:
prediction = tf.nn.sigmoid(logits)

# Define loss and optimizer
loss_op = tf.losses.hinge_loss(logits,prediction)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

prediction = tf.nn.sigmoid(logits)
tf.summary.histogram("sigmoid", prediction)
# Define loss and optimizer
with tf.name_scope("train"):
    loss_op = tf.losses.hinge_loss(logits,prediction)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
# Evaluate model (with test logits, for dropout to be disabled)
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar("accuracy_sigmoid_GradientDescent", accuracy)

<tf.Tensor 'accuracy_sigmoid_GradientDescent:0' shape=() dtype=string>

In [45]:
start_training()

Step 1, Minibatch Loss= 0.9043, Training Accuracy= 0.053
Step 100, Minibatch Loss= 0.9043, Training Accuracy= 0.050
Step 200, Minibatch Loss= 0.9042, Training Accuracy= 0.050
Step 300, Minibatch Loss= 0.9042, Training Accuracy= 0.060
Step 400, Minibatch Loss= 0.9042, Training Accuracy= 0.060
Step 500, Minibatch Loss= 0.9041, Training Accuracy= 0.067
Step 600, Minibatch Loss= 0.9041, Training Accuracy= 0.083
Step 700, Minibatch Loss= 0.9055, Training Accuracy= 0.103
Step 800, Minibatch Loss= 0.9040, Training Accuracy= 0.107
Step 900, Minibatch Loss= 0.9040, Training Accuracy= 0.053
Step 1000, Minibatch Loss= 0.9039, Training Accuracy= 0.080
Optimization Finished!
Testing Accuracy: 0.068023376


Accuracy changed for GradientDescent and AdaDelta but was same for Adagrad optimizer.

# Part F

In [46]:
# epochs = 200
# batch size = 300
# num of hidden features = 1000
# activation function = sigmoid
# cost function = hinge loss
# gradient estimator = GradientDescent

In [47]:
for i in range(50,500,50):
    print("Num of features: %i" % i)
    num_hidden=i
    start_training(False)
    print(30 *"-")

Num of features: 50
Testing Accuracy: 0.9329625
------------------------------
Num of features: 100
Testing Accuracy: 0.06703753
------------------------------
Num of features: 150
Testing Accuracy: 0.06703753
------------------------------
Num of features: 200
Testing Accuracy: 0.068023376
------------------------------
Num of features: 250
Testing Accuracy: 0.068023376
------------------------------
Num of features: 300
Testing Accuracy: 0.9322583
------------------------------
Num of features: 350
Testing Accuracy: 0.06703753
------------------------------
Num of features: 400
Testing Accuracy: 0.9329625
------------------------------
Num of features: 450
Testing Accuracy: 0.06703753
------------------------------


The num of hidden features does affect the accuracy. Accuracy is low for any value below 200 but increases on 200 and stays the same for values 200 and above.

## Part G

In [48]:
# xavier Initializer
weights = {
    'out':  tf.get_variable("W1", shape=[num_hidden, num_classes],
           initializer=tf.contrib.layers.xavier_initializer())
}
start_training()

Step 1, Minibatch Loss= 0.1865, Training Accuracy= 0.923
Step 100, Minibatch Loss= 0.0770, Training Accuracy= 0.930
Step 200, Minibatch Loss= 0.0000, Training Accuracy= 0.937
Step 300, Minibatch Loss= 0.0025, Training Accuracy= 0.913
Step 400, Minibatch Loss= 0.0000, Training Accuracy= 0.940
Step 500, Minibatch Loss= 0.0025, Training Accuracy= 0.953
Step 600, Minibatch Loss= 0.0000, Training Accuracy= 0.930
Step 700, Minibatch Loss= 0.0025, Training Accuracy= 0.927
Step 800, Minibatch Loss= 0.0000, Training Accuracy= 0.957
Step 900, Minibatch Loss= 0.0053, Training Accuracy= 0.930
Step 1000, Minibatch Loss= 0.0000, Training Accuracy= 0.957
Optimization Finished!
Testing Accuracy: 0.9329625


In [49]:
# zeros initializer
weights = {
    'out':  tf.get_variable("W2", shape=[num_hidden, num_classes],
           initializer=tf.zeros_initializer())
}
start_training()

Step 1, Minibatch Loss= 0.8188, Training Accuracy= 0.083
Step 100, Minibatch Loss= 0.8129, Training Accuracy= 0.047
Step 200, Minibatch Loss= 0.8099, Training Accuracy= 0.057
Step 300, Minibatch Loss= 0.8069, Training Accuracy= 0.083
Step 400, Minibatch Loss= 0.8068, Training Accuracy= 0.083
Step 500, Minibatch Loss= 0.8008, Training Accuracy= 0.057
Step 600, Minibatch Loss= 0.7978, Training Accuracy= 0.073
Step 700, Minibatch Loss= 0.7978, Training Accuracy= 0.050
Step 800, Minibatch Loss= 0.7946, Training Accuracy= 0.063
Step 900, Minibatch Loss= 0.7886, Training Accuracy= 0.073
Step 1000, Minibatch Loss= 0.7855, Training Accuracy= 0.087
Optimization Finished!
Testing Accuracy: 0.06703753


The accuracy stayed about the same for xavier initialization but changed drastically low for zero initialization.